<a href="https://colab.research.google.com/github/OliverGriffiths287/OliverGriffiths287.github.io/blob/main/football.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
import json
import pandas as pd

# Extracting the average

In [74]:
# Define the URL of the page containing the salary data
url = "https://www.capology.com/uk/premier-league/salaries/2013-2014/"

# Send a GET request to the URL to retrieve the page's HTML content
req = requests.get(url)

# Extract the text content of the page (HTML source)
page_src = req.text

# Find the starting index of the average salary section in the HTML content
# The salary information is located within a specific element containing 'accounting.formatMoney("'
start_index = page_src.index('$(\'#salaries-avg\').html(accounting.formatMoney("')

# Slice the page source starting from the location of the average salary element
page_src = page_src[start_index:]

# Split the sliced HTML content by double quotes to isolate the salary value
page_src = page_src.split('"')

# Extract the average salary value (second item in the split list) and assign it to 'avg'
avg = page_src[1]

# Print or return the average salary value
avg

'1272854'

# Parsing the Player Table

This code pulls player salary data from a webpage, isolating the relevant section in the HTML. It then splits the data by players and loops through each one to extract details like name, age, position, country, and salary. Each player’s information is stored in a dictionary, which is added to a list. This results in a structured list of player data for easy access and analysis.

In [75]:
# Extract the HTML content of the page as text
page_src = req.text

# Split the page source to isolate the JSON-like data within 'var data = ...'
# This step assumes there's only one occurrence of 'var data =' in the page source
page_src = page_src.split("var data =")[1]

# Further split the result to exclude any trailing characters after the JSON data
# We're aiming to end the data at the first occurrence of ";\n"
page_src = page_src.split(";\n")[0]

# Clean the extracted data by removing the outer square brackets to prepare it for parsing
page_src = page_src.replace("[", "").replace("]", "")

# Display the cleaned data
page_src

# Split the data into individual player records, using '{' as the delimiter
# Each entry in 'players' represents data for one player
players = page_src.split("{")

# Initialize an empty list to store each player's parsed data as dictionaries
players_data = []

# Loop through each player entry in 'players' to parse their information
for player_text in players:
    # Create an empty dictionary to store parsed fields for this player
    parsed_player_data = {}

    # Remove the trailing "}," from the player data for easier parsing
    player_text = player_text.replace("},", "")

    # Replace any double newlines with single newlines to standardize formatting
    player_text = player_text.replace("\n\n", "\n")

    # Split the player's data into individual fields based on newlines
    player_fields = player_text.split("\n")

    # Uncomment the following line to print each player's text if needed
    # print(player_text)

    # Loop through each field in the player's data to extract key-value pairs
    for player_field in player_fields:
        # Skip empty lines
        if len(player_field) == 0:
            continue
        # Extract name, URL, and flag image URL if the field contains 'name'
        elif 'name' in player_field:
            # Get the link
            parsed_player_data['url'] = player_field.split("href='")[1].split("'")[0]
            # Get the flag image URL
            parsed_player_data['flag_img_url'] = player_field.split("<img src='")[1].split("'")[0]
            # Extract the player's name
            parsed_player_data['name'] = player_field.split("loading='lazy'>")[1].split("<")[0]
        # Extract monetary values if the field contains 'accounting.formatMoney'
        elif "accounting.formatMoney" in player_field:
            # Get the field key and corresponding salary value
            field_key = player_field.split("'")[1].split("'")[0]
            value = player_field.split('"')[1].split('"')[0]
            parsed_player_data[field_key] = value
        # Extract position if 'position' is in the field
        elif "position" in player_field:
            parsed_player_data['position'] = player_field.split('"')[1].split('"')[0]
        # Extract age if 'age' is in the field
        elif "age" in player_field:
            parsed_player_data["age"] = player_field.split('"')[1].split('"')[0]
        # Extract country if 'country' is in the field
        elif "country" in player_field:
            parsed_player_data["country"] = player_field.split('"')[1].split('"')[0]

    # Append the parsed player data dictionary to the list of all players' data
    players_data.append(parsed_player_data)

# Display the final parsed data for the players
players_data

[{},
 {'url': '/player/luis-suarez-31801/',
  'flag_img_url': 'https://capology-e6a3.kxcdn.com/static/images/flags/uruguay.svg',
  'name': 'Luis Suárez',
  'weekly_gross_eur': '12387932',
  'annual_gross_eur': '12387932',
  'adjusted_gross_eur': '14977429',
  'weekly_gross_gbp': '10400000',
  'annual_gross_gbp': '10400000',
  'adjusted_gross_gbp': '12573952',
  'weekly_gross_usd': '13473017',
  'annual_gross_usd': '13473017',
  'adjusted_gross_usd': '16289334',
  'weekly_net_eur': '6813363',
  'annual_net_eur': '6813363',
  'adjusted_net_eur': '8237586',
  'weekly_net_gbp': '5720000',
  'annual_net_gbp': '5720000',
  'adjusted_net_gbp': '6915674',
  'weekly_net_usd': '7410159',
  'annual_net_usd': '7410159',
  'adjusted_net_usd': '8959134',
  'position': 'F',
  'age': '26',
  'country': 'Uruguay'},
 {'url': '/player/yaya-toure-30449/',
  'flag_img_url': 'https://capology-e6a3.kxcdn.com/static/images/flags/cote-d-ivoire.svg',
  'name': 'Yaya Touré',
  'weekly_gross_eur': '11768535',
  '

# Loading it with Pandas

In [38]:

df25 = pd.DataFrame(players_data)
df25 = df25[["name", "annual_gross_gbp"]]

df25['annual_gross_gbp'] = pd.to_numeric(df25["annual_gross_gbp"], errors="coerce")

df25 = df25.query("annual_gross_gbp > 0")

df25

,name,annual_gross_gbp
1,Kevin De Bruyne,20800000.0
2,Erling Haaland,19500000.0
3,Casemiro,18200000.0
4,Mohamed Salah,18200000.0
5,Bruno Fernandes,15600000.0
...,...,...
526,Zach Abbott,156000.0
527,Paris Maghoma,156000.0
528,Imari Samuels,156000.0
529,Ryan Trevitt,130000.0


In [42]:

df24 = pd.DataFrame(players_data)
df24 = df24[["name", "annual_gross_gbp"]]

df24['annual_gross_gbp'] = pd.to_numeric(df24["annual_gross_gbp"], errors="coerce")

df24 = df24.query("annual_gross_gbp > 0")

df24

,name,annual_gross_gbp
1,Kevin De Bruyne,20800000.0
2,Erling Haaland,19500000.0
3,Casemiro,18200000.0
4,Mohamed Salah,18200000.0
5,Raphaël Varane,17680000.0
...,...,...
585,Jordan Beyer,156000.0
586,Elliot Thorpe,130000.0
587,Ryan Trevitt,130000.0
588,Gavin Kilkenny,78000.0


In [45]:

df23 = pd.DataFrame(players_data)
df23 = df23[["name", "annual_gross_gbp"]]

df23['annual_gross_gbp'] = pd.to_numeric(df23["annual_gross_gbp"], errors="coerce")

df23 = df23.query("annual_gross_gbp > 0")

df23

,name,annual_gross_gbp
1,Cristiano Ronaldo,26780000.0
2,Kevin De Bruyne,20800000.0
3,Erling Haaland,19500000.0
4,David de Gea,19500000.0
5,Casemiro,18200000.0
...,...,...
587,Chem Campbell,130000.0
588,Jordan Smith,130000.0
589,Cody Drameh,130000.0
590,Leo Fuhr Hjelde,104000.0


In [48]:

df22 = pd.DataFrame(players_data)
df22 = df22[["name", "annual_gross_gbp"]]

df22['annual_gross_gbp'] = pd.to_numeric(df22["annual_gross_gbp"], errors="coerce")

df22 = df22.query("annual_gross_gbp > 0")

df22

,name,annual_gross_gbp
1,Cristiano Ronaldo,26780000.0
2,Kevin De Bruyne,20800000.0
3,David de Gea,19500000.0
4,Raphaël Varane,17680000.0
5,Romelu Lukaku,16900000.0
...,...,...
558,Sean Longstaff,52000.0
559,Kwadwo Baah,50000.0
560,Liam McCarron,40000.0
561,Flynn Clarke,40000.0


In [51]:

df21 = pd.DataFrame(players_data)
df21 = df21[["name", "annual_gross_gbp"]]

df21['annual_gross_gbp'] = pd.to_numeric(df21["annual_gross_gbp"], errors="coerce")

df21 = df21.query("annual_gross_gbp > 0")

df21

,name,annual_gross_gbp
1,Gareth Bale,31200000.0
2,Kevin De Bruyne,20800000.0
3,David de Gea,19500000.0
4,Mesut Özil,18200000.0
5,Raheem Sterling,15600000.0
...,...,...
589,Leif Davis,30000.0
590,Dan Langley,26000.0
591,Zak Brunt,26000.0
592,Brandon Pierrick,25000.0


In [58]:

df20 = pd.DataFrame(players_data)
df20 = df20[["name", "annual_gross_gbp"]]

df20['annual_gross_gbp'] = pd.to_numeric(df22["annual_gross_gbp"], errors="coerce")

df20 = df20.query("annual_gross_gbp > 0")

df20

,name,annual_gross_gbp
1,David de Gea,26780000.0
2,Mesut Özil,20800000.0
3,Kevin De Bruyne,19500000.0
4,Raheem Sterling,17680000.0
5,N&#39;Golo Kanté,16900000.0
...,...,...
553,Sean Longstaff,100000.0
554,Enzo Loiodice,90000.0
555,Matthew Longstaff,80000.0
556,Andrew Crofts,80000.0


In [61]:

df19 = pd.DataFrame(players_data)
df19 = df19[["name", "annual_gross_gbp"]]

df19['annual_gross_gbp'] = pd.to_numeric(df19["annual_gross_gbp"], errors="coerce")

df19 = df19.query("annual_gross_gbp > 0")

df19

,name,annual_gross_gbp
1,Alexis Sánchez,18200000.0
2,Mesut Özil,18200000.0
3,Kevin De Bruyne,18200000.0
4,Gonzalo Higuaín,15777000.0
5,Paul Pogba,15080000.0
...,...,...
634,Lewis Cass,20800.0
635,Aaron Rowe,20800.0
636,Cameron John,20800.0
637,Ki-Jana Hoever,20800.0


In [65]:

df18 = pd.DataFrame(players_data)
df18 = df18[["name", "annual_gross_gbp"]]

df18['annual_gross_gbp'] = pd.to_numeric(df18["annual_gross_gbp"], errors="coerce")

df18 = df18.query("annual_gross_gbp > 0")
df18 = df18.drop_duplicates(subset=["name"])

df18

,name,annual_gross_gbp
1,Mesut Özil,18200000.0
2,Alexis Sánchez,18200000.0
4,Eden Hazard,11589000.0
5,Cesc Fàbregas,10694000.0
6,Paul Pogba,10400000.0
...,...,...
593,Grady Diangana,20800.0
594,Tyrese Campbell,20800.0
595,Joy Mukena,20800.0
596,Ethan Ampadu,20800.0


In [67]:
df17 = pd.DataFrame(players_data)
df17 = df17[["name", "annual_gross_gbp"]]

df17['annual_gross_gbp'] = pd.to_numeric(df17["annual_gross_gbp"], errors="coerce")

df17 = df17.query("annual_gross_gbp > 0")
df17 = df17.drop_duplicates(subset=["name"])

df17

,name,annual_gross_gbp
1,Mesut Özil,18200000.0
2,Alexis Sánchez,18200000.0
4,Eden Hazard,11589000.0
5,Cesc Fàbregas,10694000.0
6,Paul Pogba,10400000.0
...,...,...
593,Grady Diangana,20800.0
594,Tyrese Campbell,20800.0
595,Joy Mukena,20800.0
596,Ethan Ampadu,20800.0


In [70]:
df16 = pd.DataFrame(players_data)
df16 = df16[["name", "annual_gross_gbp"]]

df16['annual_gross_gbp'] = pd.to_numeric(df16["annual_gross_gbp"], errors="coerce")

df16 = df16.query("annual_gross_gbp > 0")
df16 = df16.drop_duplicates(subset=["name"])

df16

<ipython-input-70-74ed172544e7>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df16['annual_gross_gbp'] = pd.to_numeric(df16["annual_gross_gbp"], errors="coerce")


,name,annual_gross_gbp
1,Bastian Schweinsteiger,12480000.0
2,Wayne Rooney,12220000.0
3,David de Gea,10400000.0
4,Yaya Touré,9880000.0
5,Eden Hazard,9286000.0
...,...,...
634,Jordan Lee,20800.0
635,Ben Chilwell,20800.0
636,Rushian Hepburn-Murphy,20800.0
637,Ro-Shaun Williams,20800.0


In [73]:
df15 = pd.DataFrame(players_data)
df15 = df15[["name", "annual_gross_gbp"]]

df15['annual_gross_gbp'] = pd.to_numeric(df15["annual_gross_gbp"], errors="coerce")

df15 = df15.query("annual_gross_gbp > 0")
df15 = df15.drop_duplicates(subset=["name"])

df15

,name,annual_gross_gbp
1,Falcao,14820000.0
2,Ángel Di María,13000000.0
3,Wayne Rooney,12220000.0
4,Robin van Persie,10920000.0
5,Yaya Touré,9880000.0
...,...,...
610,Angeliño,20800.0
611,Kieran Dowell,20800.0
612,Oliver Shenton,20800.0
613,Bersant Celina,20800.0


In [76]:
df14 = pd.DataFrame(players_data)
df14 = df14[["name", "annual_gross_gbp"]]

df14['annual_gross_gbp'] = pd.to_numeric(df14["annual_gross_gbp"], errors="coerce")

df14 = df14.query("annual_gross_gbp > 0")
df14 = df14.drop_duplicates(subset=["name"])

df14

,name,annual_gross_gbp
1,Luis Suárez,10400000.0
2,Yaya Touré,9880000.0
3,Eden Hazard,9449000.0
4,Robin van Persie,9360000.0
5,Wayne Rooney,9360000.0
...,...,...
587,Ryan Ledson,20800.0
588,George Williams,20800.0
589,Leo Chambers,20800.0
590,Hiram Boateng,20800.0


In [77]:
import pandas as pd

# Add a 'season' column to each dataframe
df25['season'] = '2024-2025'
df24['season'] = '2023-2024'
df23['season'] = '2022-2023'
df22['season'] = '2021-2022'
df21['season'] = '2020-2021'
df20['season'] = '2019-2020'
df19['season'] = '2018-2019'
df18['season'] = '2017-2018'
df17['season'] = '2016-2017'
df16['season'] = '2015-2016'
df15['season'] = '2014-2015'
df14['season'] = '2013-2014'

# Combine all dataframes into a list
dfs = [df25, df24, df23, df22, df21, df20, df19, df18, df17, df16, df15, df14]

# Concatenate all dataframes in the list
merged_df = pd.concat(dfs, ignore_index=True)

# If you want to sort by season
merged_df = merged_df.sort_values('season', ascending=False)

# Display the first few rows of the merged dataframe
print(merged_df.head())

# Save the merged dataframe to a CSV file (optional)
merged_df.to_csv('merged_data.csv', index=False)

                      name  annual_gross_gbp     season
0          Kevin De Bruyne        20800000.0  2024-2025
349          Harry Toffolo         1820000.0  2024-2025
363                 Danilo         1560000.0  2024-2025
362  Crysencio Summerville         1560000.0  2024-2025
361             Ramón Sosa         1560000.0  2024-2025


In [10]:
import requests
import pandas as pd

def extract_average_salary(url):
    req = requests.get(url)
    page_src = req.text

    try:
        # Find the starting index of the average salary section in the HTML content
        start_index = page_src.index('$(\'#salaries-avg\').html(accounting.formatMoney("')

        # Slice the page source starting from the location of the average salary element
        page_src = page_src[start_index:]

        # Split the sliced HTML content by double quotes to isolate the salary value
        page_src = page_src.split('"')

        # Extract the average salary value (second item in the split list)
        avg = page_src[1]

        # Convert the string to a float, removing commas
        return float(avg.replace(',', ''))
    except ValueError:
        print(f"Could not find average salary data for {url}")
        return None

# Define a list of seasons
seasons = ["2024-2025", "2023-2024", "2022-2023", "2021-2022", "2020-2021", "2019-2020", "2018-2019", "2017-2018", "2016-2017",
           "2015-16", "2014-2015", "2013-2014"]

# Base URL of the website
base_url = "https://www.capology.com/uk/premier-league/salaries/"

# Create a list to store the results
results = []

# Loop through the seasons and extract average salaries
for season in seasons:
    url = base_url + season
    avg_salary = extract_average_salary(url)
    if avg_salary is not None:
        results.append({"Season": season, "Average Salary": avg_salary})

# Create a dataframe from the results
df = pd.DataFrame(results)

# Display the dataframe
print(df)

df.to_csv("PL_avg_salaries.csv", index=False)

Could not find average salary data for https://www.capology.com/uk/premier-league/salaries/2015-16
       Season  Average Salary
0   2024-2025       3521900.0
1   2023-2024       3167534.0
2   2022-2023       3523096.0
3   2021-2022       3243944.0
4   2020-2021       2928754.0
5   2019-2020       2813526.0
6   2018-2019       2098981.0
7   2017-2018       2031578.0
8   2016-2017       1687701.0
9   2014-2015       1502694.0
10  2013-2014       1272854.0


In [ ]:
import requests
import pandas as pd

def extract_players_data(url):
    req = requests.get(url)
    page_src = req.text

    # Split and clean the page source to isolate the JSON-like data
    page_src = page_src.split("var data =")[1].split(";\n")[0]
    page_src = page_src.replace("[", "").replace("]", "")

    # Split the data into individual player records
    players = page_src.split("{")

    players_data = []

    for player_text in players:
        parsed_player_data = {}
        player_text = player_text.replace("},", "").replace("\n\n", "\n")
        player_fields = player_text.split("\n")

        for player_field in player_fields:
            if len(player_field) == 0:
                continue
            elif 'name' in player_field:
                parsed_player_data['name'] = player_field.split("loading='lazy'>")[1].split("<")[0]
            elif "accounting.formatMoney" in player_field and "'annual'" in player_field:
                value = player_field.split('"')[1].split('"')[0]
                parsed_player_data['annual_salary'] = float(value.replace(',', ''))

        if parsed_player_data:  # Only append if we have data
            players_data.append(parsed_player_data)

    return players_data

# Define a list of seasons
seasons = ["2022-2023", "2021-2022", "2020-2021", "2019-2020", "2018-2019"]

# Base URL of the website
base_url = "https://www.capology.com/uk/premier-league/salaries/"

# Create a list to store all players' data
all_players_data = []

# Loop through the seasons and extract players' data
for season in seasons:
    url = base_url + season
    players_data = extract_players_data(url)

    # Add season information to each player's data
    for player in players_data:
        player['season'] = season

    all_players_data.extend(players_data)

# Create a dataframe from all players' data
df = pd.DataFrame(all_players_data)

# Reorder columns to have season first
df = df[['season', 'name', 'annual_salary']]

# Display the first few rows of the dataframe
print(df.head())

# Display basic statistics of the dataframe
print(df.describe())

# Save the dataframe to a CSV file (optional)
df.to_csv('premier_league_salaries.csv', index=False)